In [1]:
from rnamodif.architectures.rodan_pretrained_modcaller import RodanPretrainedModcaller
from rnamodif.data_utils.dataloading2 import nanopore_datamodule
from rnamodif.data_utils.split_methods import get_kfold_splits, get_fullvalid_split, get_valid_portions
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import ModelCheckpoint

model = RodanPretrainedModcaller(lr=1e-4, use_Ns_for_loss=False, weighted_loss=False, use_basecalling_loss=True)
dm = nanopore_datamodule(
    splits = get_kfold_splits(
        pos_exps=['m6A_33_covid'],
        neg_exps=['m6A_0_covid'], 
        total_k=5, 
        current_k=0
    )
    +get_fullvalid_split()(pos_files=['m6a_novoa_1', 'm6a_novoa_2', 'm6A_5_covid','m6A_10_covid'], neg_files=['UNM_novoa_1', 'UNM_novoa_2', 'UNM_novoa_short']),
    verbose=1, 
    batch_size=32, 
    valid_limit=200, 
    workers=16,
    window=4096,
    normalization='rodan',
)

experiment_name = 'm6a_0v33_base_unweighted_noN_basecalling'
checkpoint_callback = ModelCheckpoint(
    dirpath=f"/home/jovyan/RNAModif/rnamodif/checkpoints_pl/{experiment_name}", 
    save_top_k=2, 
    monitor="valid_loss", 
    save_last=True, 
    save_weights_only=False
)

logger = CometLogger(api_key="TEVQbgxxvilM1WdTyqZLJ57ac", project_name='RNAModif', experiment_name=experiment_name) 
trainer= pl.Trainer(
    max_steps = 1000000, logger=logger, accelerator='gpu',
    auto_lr_find=False, val_check_interval=500,  
    log_every_n_steps=500, benchmark=True, precision=16,#)#,
    callbacks=[checkpoint_callback],#)
    resume_from_checkpoint='/home/jovyan/RNAModif/rnamodif/checkpoints_pl/m6a_0v33_base_unweighted_noN_basecalling/last.ckpt')


trainer.fit(model, dm)

CometLogger will be initialized in online mode
Using 16bit native Automatic Mixed Precision (AMP)
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:52: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v1.7. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:737: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  ckpt_path = ckpt_path or self.resume_from_checkpoint
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-pack